# PowerBI Dataset | PowerBI データセット

このノートブックは、`Power BI Dataset`と対話するエージェントを紹介しています。エージェントはデータセットに関する一般的な質問に答えるだけではなく、エラーからの回復も行います。

> This notebook showcases an agent interacting with a `Power BI Dataset`. The agent is answering more general questions about a dataset, as well as recover from errors.

このエージェントは現在積極的に開発中であるため、全ての回答が正しいわけではありません。このエージェントは[executequery endpoint](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/execute-queries)に対して動作しますが、削除は許可されていません。

> Note that, as this agent is in active development, all answers might not be correct. It runs against the [executequery endpoint](https://learn.microsoft.com/en-us/rest/api/power-bi/datasets/execute-queries), which does not allow deletes.

### Notes: | 注意事項：

* この機能は、`pip install azure-identity`でインストールできるazure.identityパッケージによる認証に依存しています。あるいは、資格情報を提供せずに、トークンを文字列として使用してPower BIデータセットを作成することも可能です。

  > It relies on authentication with the azure.identity package, which can be installed with `pip install azure-identity`. Alternatively you can create the powerbi dataset with a token as a string without supplying the credentials.

* RLSが有効になっているデータセットを使用する際に、なりすまし用のユーザー名を指定することもできます。

  > You can also supply a username to impersonate for use with datasets that have RLS enabled.

* このツールキットは、質問からクエリを生成するためにLLMを使用し、エージェントは全体的な実行のためにLLMを使用します。

  > The toolkit uses a LLM to create the query from the question, the agent uses the LLM for the overall execution.

* テストは主に `gpt-3.5-turbo-instruct` モデルで行われましたが、codex モデルはあまりうまく機能しないようでした。

  > Testing was done mostly with a `gpt-3.5-turbo-instruct` model, codex models did not seem to perform ver well.





## Initialization | 初期化




In [ ]:
from azure.identity import DefaultAzureCredential
from langchain.agents.agent_toolkits import PowerBIToolkit, create_pbi_agent
from langchain.chat_models import ChatOpenAI
from langchain.utilities.powerbi import PowerBIDataset

In [ ]:
fast_llm = ChatOpenAI(
    temperature=0.5, max_tokens=1000, model_name="gpt-3.5-turbo", verbose=True
)
smart_llm = ChatOpenAI(temperature=0, max_tokens=100, model_name="gpt-4", verbose=True)

toolkit = PowerBIToolkit(
    powerbi=PowerBIDataset(
        dataset_id="<dataset_id>",
        table_names=["table1", "table2"],
        credential=DefaultAzureCredential(),
    ),
    llm=smart_llm,
)

agent_executor = create_pbi_agent(
    llm=fast_llm,
    toolkit=toolkit,
    verbose=True,
)

## Example: describing a table | 例：表の説明




In [ ]:
agent_executor.run("Describe table1")

## Example: simple query on a table | 例：テーブルに対するシンプルなクエリ

この例では、エージェントが実際にテーブルの行数を取得するための正しいクエリを見つけ出します。

> In this example, the agent actually figures out the correct query to get a row count of the table.




In [ ]:
agent_executor.run("How many records are in table1?")

## Example: running queries | 例：クエリを実行する




In [ ]:
agent_executor.run("How many records are there by dimension1 in table2?")

In [ ]:
agent_executor.run("What unique values are there for dimensions2 in table2")

## Example: add your own few-shot prompts | 例：ご自身の数ショットプロンプトを追加してください




In [ ]:
# fictional example
few_shots = """
Question: How many rows are in the table revenue?
DAX: EVALUATE ROW("Number of rows", COUNTROWS(revenue_details))
----
Question: How many rows are in the table revenue where year is not empty?
DAX: EVALUATE ROW("Number of rows", COUNTROWS(FILTER(revenue_details, revenue_details[year] <> "")))
----
Question: What was the average of value in revenue in dollars?
DAX: EVALUATE ROW("Average", AVERAGE(revenue_details[dollar_value]))
----
"""
toolkit = PowerBIToolkit(
    powerbi=PowerBIDataset(
        dataset_id="<dataset_id>",
        table_names=["table1", "table2"],
        credential=DefaultAzureCredential(),
    ),
    llm=smart_llm,
    examples=few_shots,
)
agent_executor = create_pbi_agent(
    llm=fast_llm,
    toolkit=toolkit,
    verbose=True,
)

In [ ]:
agent_executor.run("What was the maximum of value in revenue in dollars in 2022?")